In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from url_helpers import analyze_url
from matching import *
from textstat.textstat import textstat
import urllib, bs4

pd.options.mode.chained_assignment = None

import warnings
warnings.simplefilter("ignore", category=UserWarning)

In [2]:
import subprocess
import os

title = 'Novak Djokovic stunned as Australian Open title defence ends against Denis Istomin'
os.chdir('../lib/clickbait')
a,b = subprocess.Popen(['venv/bin/python', 'src/detect.py', title], stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
os.chdir('../../src')

In [3]:
a, b

(b'headline is 0.85 % clickbaity\n', b'Using TensorFlow backend.\n')

In [ ]:
outlet1 = 'dailymail.co.uk'
outlet2 = 'nytimes.com'
outlet3 = 'theatlantic.com'
outlet4 = 'washingtonpost.com'
#DailyMail -> Bad
#NYTimes -> Neutral for acsh Top for climatefeedback
#the Atlantic -> Top for acsh Neutral for climatefeedback

In [ ]:
outlet = outlet3
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')


df[df.apply(lambda x: analyze_url(x['url'])[0]==outlet, axis=1)].to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v3.tsv', cache_dir+outlet+'.tsv', cache_dir+outlet+'.tsv')
df = pd.read_csv(cache_dir+outlet+'.tsv', sep='\t').fillna(0)

for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/exp2/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/exp2/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/exp2/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/exp2/results.tsv', sep='\t').drop('paper', axis=1).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

df['readability'] = df['full_text'].apply(lambda x: textstat.flesch_reading_ease(x))
df['title_subjectivity'] = df['title'].apply(lambda x: TextBlob(x).subjectivity)
df['title_polarity'] = df['title'].apply(lambda x: TextBlob(x).polarity)

df = pd.concat([df.drop(['quote_indicators'], axis=1), df['quote_indicators'].apply(lambda x: pd.Series(eval(x)))], axis=1)


df['has_author'] = ~(df['authors'].isnull() | (df['authors'] == ''))

G = read_graph(cache_dir+'diffusion_graph/pruned_graph_v3.tsv')
pr = nx.pagerank(G.reverse())

df['pageRank'] = df['url'].apply(lambda x: pr[x])

df['word_count'] = df['full_text'].apply(lambda x: len(re.findall(r'\w+', x)))

df['alexa_rank']=df['url'].apply(lambda x: bs4.BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+str(x)).read(), "xml").find("REACH")['RANK'])


df.url = df.url.apply(lambda x: analyze_url(x)[0])

df[['url', 'likes', 'replies_count',
    'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
    'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
    'tweets_time_delta', 'users_countries', 'users_median_followers',
    'users_median_friends', 'users_median_tweets', 'related', 'readability',
    'title_subjectivity', 'title_polarity', 'count_all_quotes',
    'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
    'has_author', 'pageRank', 'word_count', 'alexa_rank']].to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

In [ ]:
df = pd.concat([pd.read_csv(cache_dir+outlet1+'.tsv', sep='\t'), pd.read_csv(cache_dir+outlet2+'.tsv', sep='\t'), pd.read_csv(cache_dir+outlet3+'.tsv', sep='\t')])
y = df[['url']].values
X = np.array(df.drop('url', axis=1).values, dtype=np.float32)
X = (X - X.mean(axis=0)) / (X.std(axis=0)+1e-9)

In [ ]:
# n_est = 100
# m_dep = 10
# classifier = RandomForestClassifier(n_estimators=n_est, max_depth=m_dep, n_jobs=-1)
# classifier.fit(X, y)
# result = classifier.feature_importances_

In [ ]:
from sklearn.feature_selection import f_classif
_, result = f_classif(X,y)

d = {c:r for c, r in zip(df.columns[1:], result)}
sorted(d.items(), key=lambda kv: kv[1])